In [9]:
# import itertools
# import os
# import numpy as np
# import torch
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import glob
# import json
# import logging
# import sys
# import torch.nn as nn
# import torch.optim as optim
# import random
# def batch_equalizer_fn(args):
#     eeg = args[0]
# #     print("eegshape=",eeg.shape)
#     num_stimuli = len(args) - 1
#     # repeat eeg num_stimuli times
#     new_eeg = torch.cat([eeg] * num_stimuli, dim=0)
#     all_features = [new_eeg]
# #     print("all_features=",all_features[0].shape)

#     # create args
#     args_to_zip = [args[i::num_stimuli] for i in range(1, num_stimuli + 1)]
# #     print("args_to_zip=",args_to_zip[0].shape,args_to_zip[1].shape)

#     for stimuli_features in zip(*args_to_zip):
# #         print("stimuli_features=",stimuli_features[0].shape,stimuli_features[1].shape)
#         for i in range(num_stimuli):
#             shift = i
#             shifted_tuple = tuple(stimuli_features[(j - shift) % len(stimuli_features)] for j in range(len(stimuli_features)))
#             stimulus_rolled = torch.stack(shifted_tuple)
# #             print("stimulus_rolled=", stimulus_rolled.shape)
#             # reshape stimulus_rolled to merge the first two dimensions
#             stimulus_rolled = stimulus_rolled.view(-1, stimulus_rolled.size(2), stimulus_rolled.size(3))
# #             print("stimulus_rolled1=", stimulus_rolled.shape)

#             all_features.append(stimulus_rolled)
# #     print("all_features1=",all_features)
    
#     labels_list = [torch.tensor([[1 if ii == i else 0 for ii in range(num_stimuli)]]) for i in range(num_stimuli)]
#     labels = torch.cat([label.repeat(eeg.size(0), 1) for label in labels_list], dim=0)
# #     print("labels=",labels)
# #     print("tuple(all_features)=", tuple(all_features))

#     return tuple(all_features), labels

# def shuffle_fn(args, number_mismatch):
#     # repeat the last argument number_mismatch times
#     args = list(args)
#     for _ in range(number_mismatch):
#         args.append(args[-1][torch.randperm(args[-1].size(0))])
#     return tuple(args)

# # Function to create frames from a tensor
# def frame_tensor(tensor, window_length, hop_length):
#     num_frames = (tensor.size(0) - window_length) // hop_length + 1
#     frames = torch.stack(
#         [tensor[i * hop_length : i * hop_length + window_length] for i in range(num_frames)]
#     )
#     return frames

# def process_eeg(original_tensors_list):
#     reshaped_tensors = [tensor[i].view(320, 64) for tensor in original_tensors_list for i in range(tensor.size(0))]

#     lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
#     lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

#     # Shuffle the lists
#     random.shuffle(lists_of_tensors)

#     final_tensors = []
#     for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
#         concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 64)
#         final_tensors.append(concatenated_tensors)

#     return final_tensors
# def process_stimuli(original_tensors_list):
#     reshaped_tensors = [tensor[i].view(320, 1) for tensor in original_tensors_list for i in range(tensor.size(0))]

#     lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
#     lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

#     # Shuffle the lists
#     random.shuffle(lists_of_tensors)

#     final_tensors = []
#     for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
#         concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 1)
#         final_tensors.append(concatenated_tensors)

#     return final_tensors
# class PyTorchDataGenerator(Dataset):
#     def __init__(self, files, window_length):
#         self.window_length = window_length
#         self.files = self.group_recordings(files)

#     def group_recordings(self, files):
#         new_files = []
#         grouped = itertools.groupby(
#             sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3])
#         )
#         for recording_name, feature_paths in grouped:
#             new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
# #         print("new_files=", new_files)
#         return new_files
    

#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, recording_index):
#         data = []
#         for feature in self.files[recording_index]:
#             f = np.load(feature).astype(np.float32)
# #             print("f_before=", f.shape)
#             if f.ndim == 1:
#                 f = f[:, None]
# #                 print("f_after=", f.shape)
#             data += [f]
# #         print("data_before=", data)
#         data = self.prepare_data(data)
# #         print("data_after=", data)
# #         print("tuple(torch.tensor(x) for x in data)=",tuple(torch.tensor(x) for x in data))
#         return tuple(torch.tensor(x) for x in data)

#     def __call__(self):
#         for idx in range(self.__len__()):
#             yield self.__getitem__(idx)

#             if idx == self.__len__() - 1:
#                 self.on_epoch_end()

#     def on_epoch_end(self):
#         np.random.shuffle(self.files)

#     def prepare_data(self, data):
#         # make sure data has dimensionality of (n_samples, n_features)
#         return data

# def create_pytorch_dataset(
#     data_generator,
#     window_length,
#     batch_equalizer_fn=None,
#     frame_tensor=None,
#     process_eeg=None,
#     process_stimuli=None,
#     hop_length=64,
#     batch_size=64,
#     number_mismatch=None,
#     data_types=(torch.float32, torch.float32),
#     feature_dims=(64, 1)
# ):
#     dataset = data_generator
#     if frame_tensor is not None:
#         i=0
#         for data in dataset:
#             dataset = [(frame_tensor(data[0], window_length, hop_length),frame_tensor(data[1], window_length, hop_length),)]


#     if number_mismatch is not None:
#         # map second argument to shifted version
#         dataset = [
#         shuffle_fn(data, number_mismatch) for data in dataset
#     ]
    
#     if process_eeg is not None and process_stimuli is not None:
#         # map second argument to shifted version
#         dataset=[process_eeg([data[0] for data in dataset]),
#         process_stimuli([data[1] for data in dataset]),
#         process_stimuli([data[2] for data in dataset]),
#         process_stimuli([data[3] for data in dataset]),
#         process_stimuli([data[4] for data in dataset]),
#         process_stimuli([data[5] for data in dataset])]
#         dataset = [tuple([dataset[0][i],dataset[1][i],dataset[2][i],dataset[3][i],dataset[4][i],dataset[5][i]]) for i in range(len(dataset[0]))]
# #         print(dataset[0][0].shape,dataset[0][1].shape,dataset[0][2].shape)

#     if batch_equalizer_fn is not None:
#         # Create the labels and make sure classes are balanced
#         dataset = [
#             tuple(batch_equalizer_fn(args)) for args in dataset
#         ]

#     return tuple(dataset)

# window_length_s = 5
# fs = 64

# window_length = window_length_s * fs  # 5 seconds
# # Hop length between two consecutive decision windows
# hop_length = 64

# epochs = 100
# patience = 5
# batch_size = 64 #fixed in the code
# only_evaluate = True
# number_mismatch = 4 # or 4



# training_log_filename = "training_log_{}_{}.csv".format(number_mismatch, window_length_s)
# data_folder = "split_data/split_data"

# # stimulus feature which will be used for training the model. Can be either 'envelope' ( dimension 1) or 'mel' (dimension 28)
# stimulus_features = ["envelope"]
# stimulus_dimension = 1

# # uncomment if you want to train with the mel spectrogram stimulus representation
# # stimulus_features = ["mel"]
# # stimulus_dimension = 10

# features = ["eeg"] + stimulus_features
# # print("features=", features)
# train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# # Create list of numpy array files
# train_generator = PyTorchDataGenerator(train_files, window_length)
# import pdb
# dataset_train = create_pytorch_dataset(train_generator, window_length, batch_equalizer_fn,frame_tensor,process_eeg,process_stimuli,
#                                   hop_length, batch_size,
#                                   number_mismatch=number_mismatch,
#                                   data_types=(torch.float32, torch.float32),
#                                   feature_dims=(64, stimulus_dimension))

# val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# val_generator = PyTorchDataGenerator(val_files, window_length)
# dataset_val = create_pytorch_dataset(val_generator,  window_length, batch_equalizer_fn,frame_tensor,process_eeg,process_stimuli,
#                                   hop_length, batch_size,
#                                   number_mismatch=number_mismatch,
#                                   data_types=(torch.float32, torch.float32),
#                                   feature_dims=(64, stimulus_dimension))



In [2]:
import itertools
import os
import numpy as np
import torch
import tqdm
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import glob
import json
import logging
import sys
import torch.nn as nn
import torch.optim as optim
import random
def batch_equalizer_fn(args):
    eeg = args[0]
#     print("eegshape=",eeg.shape)
    num_stimuli = len(args) - 1
    # repeat eeg num_stimuli times
    new_eeg = torch.cat([eeg] * num_stimuli, dim=0)
    all_features = [new_eeg]
#     print("all_features=",all_features[0].shape)

    # create args
    args_to_zip = [args[i::num_stimuli] for i in range(1, num_stimuli + 1)]
#     print("args_to_zip=",args_to_zip[0].shape,args_to_zip[1].shape)

    for stimuli_features in zip(*args_to_zip):
#         print("stimuli_features=",stimuli_features[0].shape,stimuli_features[1].shape)
        for i in range(num_stimuli):
            shift = i
            shifted_tuple = tuple(stimuli_features[(j - shift) % len(stimuli_features)] for j in range(len(stimuli_features)))
#             print(len(shifted_tuple))
            stimulus_rolled = torch.stack(shifted_tuple)
#             print("stimulus_rolled=", stimulus_rolled.shape)
            # reshape stimulus_rolled to merge the first two dimensions
            stimulus_rolled = stimulus_rolled.view(-1, stimulus_rolled.size(2), stimulus_rolled.size(3))
#             print("stimulus_rolled1=", stimulus_rolled.shape)

            all_features.append(stimulus_rolled)
#     print("all_features1=",all_features)
    
    labels_list = [torch.tensor([[1 if ii == i else 0 for ii in range(num_stimuli)]]) for i in range(num_stimuli)]
    labels = torch.cat([label.repeat(eeg.size(0), 1) for label in labels_list], dim=0)
#     print("labels=",labels)
#     print("tuple(all_features)=", tuple(all_features))

    return tuple(all_features), labels

def shuffle_fn(args, number_mismatch):
    # repeat the last argument number_mismatch times
    args = list(args)
    for _ in range(number_mismatch):
        args.append(args[-1][torch.randperm(args[-1].size(0))])
    return tuple(args)

# Function to create frames from a tensor
def frame_tensor(tensor, window_length, hop_length):
    num_frames = (tensor.size(0) - window_length) // hop_length + 1
    frames = torch.stack(
        [tensor[i * hop_length : i * hop_length + window_length] for i in range(num_frames)]
    )
    return frames

def process_eeg(original_tensors_list):
    reshaped_tensors = [tensor[i].view(320, 64) for tensor in original_tensors_list for i in range(tensor.size(0))]

    lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
    lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

    # Shuffle the lists
    random.shuffle(lists_of_tensors)

    final_tensors = []
    for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
        concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, 64)
        final_tensors.append(concatenated_tensors)

    return final_tensors
def process_stimuli(original_tensors_list,shape):
    reshaped_tensors = [tensor[i].view(320, shape) for tensor in original_tensors_list for i in range(tensor.size(0))]

    lists_of_tensors = [reshaped_tensors[i:i+8] for i in range(0, len(reshaped_tensors), 8)]
    lists_of_tensors = lists_of_tensors[:len(reshaped_tensors)//8]

    # Shuffle the lists
    random.shuffle(lists_of_tensors)

    final_tensors = []
    for chunk_of_lists in zip(*(iter(lists_of_tensors),) * 8):
        concatenated_tensors = torch.cat([torch.unsqueeze(tensor, 0) for sublist in chunk_of_lists for tensor in sublist], dim=0).view(64, 320, shape)
        final_tensors.append(concatenated_tensors)

    return final_tensors
class PyTorchDataGenerator(Dataset):
    def __init__(self, files, window_length):
        self.window_length = window_length
        self.files = self.group_recordings(files)

    def group_recordings(self, files):
        new_files = []
        grouped = itertools.groupby(
            sorted(files), lambda x: "_-_".join(os.path.basename(x).split("_-_")[:3])
        )
        for recording_name, feature_paths in grouped:
            new_files += [sorted(feature_paths, key=lambda x: "0" if x == "eeg" else x)]
#         print("new_files=", new_files[0:2])
        return new_files
    

    def __len__(self):
        return len(self.files)

    def __getitem__(self, recording_index):
        data = []
        for feature in self.files[recording_index]:
            f = np.load(feature).astype(np.float32)
#             print("f_before=", f.shape)
            if f.ndim == 1:
                f = f[:, None]
#                 print("f_after=", f.shape)
            data += [f]
#         print("data_before=", data)
        data = self.prepare_data(data)
#         print("data_after=", data)
#         print("tuple(torch.tensor(x) for x in data)=",tuple(torch.tensor(x) for x in data))
        return tuple(torch.tensor(x) for x in data)

    def __call__(self):
        for idx in range(self.__len__()):
            yield self.__getitem__(idx)

            if idx == self.__len__() - 1:
                self.on_epoch_end()

    def on_epoch_end(self):
        np.random.shuffle(self.files)

    def prepare_data(self, data):
        # make sure data has dimensionality of (n_samples, n_features)
        return data

def create_pytorch_dataset(
    data_generator,
    window_length,
    batch_equalizer_fn=None,
    frame_tensor=None,
    process_eeg=None,
    process_stimuli=None,
    hop_length=64,
    batch_size=64,
    number_mismatch=None,
    data_types=(torch.float32, torch.float32),
    feature_dims=(64, 1)
):
    dataset = data_generator
    
    if frame_tensor is not None:
        i=0
        dataset = [(frame_tensor(data[0], window_length, hop_length),frame_tensor(data[1], window_length, hop_length)) for data in tqdm(dataset)]


    if number_mismatch is not None:
        # map second argument to shifted version
        dataset = [
        shuffle_fn(data, number_mismatch) for data in dataset
    ]
    
    if process_eeg is not None and process_stimuli is not None:
        # map second argument to shifted version
        dataset=[process_eeg([data[0] for data in tqdm(dataset)]),
        process_stimuli([data[1] for data in tqdm(dataset)],feature_dims[1])]
        dataset = [tuple([dataset[0][i],dataset[1][i]]) for i in range(len(dataset[0]))]
#         print(dataset[0][0].shape,dataset[0][1].shape,dataset[0][2].shape)

    if batch_equalizer_fn is not None:
        # Create the labels and make sure classes are balanced
        dataset = [
            tuple(batch_equalizer_fn(args)) for args in dataset
        ]

    return tuple(dataset)

window_length_s = 5
fs = 64

window_length = window_length_s * fs  # 5 seconds
# Hop length between two consecutive decision windows
hop_length = 64

epochs = 100
patience = 5
batch_size = 64 #fixed in the code
only_evaluate = True
number_mismatch = 4 # or 4



training_log_filename = "training_log_{}_{}.csv".format(number_mismatch, window_length_s)
data_folder = "split_data/split_data"

# stimulus feature which will be used for training the model. Can be either 'envelope' ( dimension 1) or 'mel' (dimension 28)
# stimulus_features = ["envelope"]
# stimulus_dimension = 1

# uncomment if you want to train with the mel spectrogram stimulus representation
stimulus_features = ["mel"]
stimulus_dimension = 10

features = ["eeg"] + stimulus_features
# print("features=", features)
train_files = [x for x in glob.glob(os.path.join(data_folder, "train_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
# Create list of numpy array files
train_generator = PyTorchDataGenerator(train_files, window_length)
import pdb
dataset_train = create_pytorch_dataset(train_generator, window_length, None,frame_tensor,process_eeg,process_stimuli,
                                  hop_length, batch_size,
                                  number_mismatch=None,
                                  data_types=(torch.float32, torch.float32),
                                  feature_dims=(64, stimulus_dimension))

val_files = [x for x in glob.glob(os.path.join(data_folder, "val_-_*")) if os.path.basename(x).split("_-_")[-1].split(".")[0] in features]
val_generator = PyTorchDataGenerator(val_files, window_length)
dataset_val = create_pytorch_dataset(val_generator,  window_length, None,frame_tensor,process_eeg,process_stimuli,
                                  hop_length, batch_size,
                                  number_mismatch=None,
                                  data_types=(torch.float32, torch.float32),
                                  feature_dims=(64, stimulus_dimension))



666


NameError: name 'tqdm' is not defined

In [3]:
c = 0 
for i in dataset_train:
    print(i)
    c += 1
    if c == 2:
        break

NameError: name 'dataset_train' is not defined

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm 

class ResNet34EEG(nn.Module):
    def __init__(self):
        super(ResNet34EEG, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.conv1 = nn.Conv2d(64, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        return self.model(x)

class ResNet34Mel(nn.Module):
    def __init__(self):
        super(ResNet34Mel, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.conv1 = nn.Conv2d(10, 64, kernel_size=7, stride=2, padding=3, bias=False)

    def forward(self, x):
        return self.model(x)


device = 'cpu'

resnet_eeg = ResNet34EEG().to(device)
resnet_mel = ResNet34Mel().to(device)


C:\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
optimizer = optim.Adam(list(resnet_eeg.parameters()) + list(resnet_mel.parameters()), lr=0.001)
batch_size = 2
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0.0
    for eeg_input, mel_input in tqdm(dataset_train, desc=f'Epoch {epoch+1}/{num_epochs}'):
        optimizer.zero_grad()
        # print(eeg_input.shape)
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        output_eeg = resnet_eeg(eeg_input)
        output_mel = resnet_mel(mel_input)

        loss = torch.abs(output_eeg - output_mel).sum()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

# Save the trained models if needed
torch.save(resnet_eeg.state_dict(), 'resnet_eeg.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel.pth')

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm  

class CustomResNet(nn.Module):
    def __init__(self, input_channels):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

resnet_eeg_soft = CustomResNet(64)
resnet_mel_soft = CustomResNet(10)

resnet_eeg_soft = resnet_eeg_soft.to(device)
resnet_mel_soft = resnet_mel_soft.to(device)


In [ ]:
optimizer = optim.Adam(list(resnet_eeg_soft.parameters()) + list(resnet_mel_soft.parameters()), lr=0.001)

batch_size = 8
num_epochs = 5

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    total_loss = 0.0
    for eeg_input, mel_input in tqdm(dataset_train, desc=f'Epoch {epoch+1}/{num_epochs}'):
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        optimizer.zero_grad()

        output_eeg = resnet_eeg_soft(eeg_input)
        output_mel = resnet_mel_soft(mel_input)

        # loss = torch.abs(output_eeg - output_mel).sum()
        loss = criterion(output_eeg,output_mel)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

torch.save(resnet_eeg.state_dict(), 'resnet_eeg_soft.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel_soft.pth')

### Speech FFR

In [ ]:
import numpy as np
from scipy.signal import hilbert, butter, filtfilt
import matplotlib.pyplot as plt

class EnvelopeExtractor:
    def __init__(self, speech_envelope, high_freq_cutoff, low_freq_cutoff, sampling_rate):        
        self.speech_envelope = speech_envelope
        self.high_freq_cutoff = high_freq_cutoff
        self.low_freq_cutoff = low_freq_cutoff
        self.sampling_rate = sampling_rate

    def design_bandpass_filter(self):
        nyquist = 0.5 * self.sampling_rate
        b, a = butter(4, [self.low_freq_cutoff / nyquist, self.high_freq_cutoff / nyquist], btype='band')
        return b, a

    def extract_envelope(self):
        b, a = self.design_bandpass_filter()
        filtered_envelope = filtfilt(b, a, self.speech_envelope)
        analytic_signal = hilbert(filtered_envelope)
        amplitude_envelope = np.abs(analytic_signal)
        return amplitude_envelope

### Attention Module

In [ ]:
import torch
import torch.nn as nn

class AttentionEncoder(nn.Module):
    def __init__(self, eeg_input_size, audio_input_size, hidden_size):
        super(AttentionEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.eeg_lstm = nn.LSTM(eeg_input_size, hidden_size, bidirectional=True)
        self.audio_lstm = nn.LSTM(audio_input_size, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 4, hidden_size)  # Output size matches hidden size
        self.attention = nn.Linear(hidden_size * 4, 1, bias=False)

    def forward(self, eeg_input, audio_input):
        eeg_outputs, (eeg_hidden, eeg_cell) = self.eeg_lstm(eeg_input)
        audio_outputs, (audio_hidden, audio_cell) = self.audio_lstm(audio_input)
        
        combined_outputs = torch.cat((eeg_outputs, audio_outputs), dim=2)
        energy = self.attention(combined_outputs)
        
        # Apply attention independently for each sequence
        attention_weights = torch.softmax(energy, dim=0)
        context_vector = torch.sum(combined_outputs * attention_weights, dim=0)
        
        # Apply a fully connected layer for the final output
        output = torch.tanh(self.fc(context_vector))
        
        return output



# # Example usage:
# eeg_data = torch.randn(10, 32, 64)  # EEG data shape: (seq_len, batch_size, input_size)
# audio_data = torch.randn(1, 32, 128)  # Audio data shape: (seq_len, batch_size, input_size)
# input_size_eeg = 64
# input_size_audio = 128
# hidden_size = 128
# # Repeat audio data to match EEG sequence length
# audio_data = audio_data.expand(10, -1, -1)  # Expand audio to (10, 32, 128)
# ae = AttentionEncoder(input_size_eeg, input_size_audio, hidden_size)
# output = ae(eeg_data, audio_data)
# print(output.shape)  #the output shape is (32,128) which is the same as the batch size and hidden size


### Speech FFR + Resnet
Shivang please create the new ffr dataset in the same format as the original dataset

In [ ]:
#APPLY BELOW CODE TO ENTIRE AUDIO DATASET THEN APPLY RESNET AS BEFORE

# extractor = EnvelopeExtractor(speech_envelope=speech_envelope,
#                               high_freq_cutoff=4000,   #freq values according to ChatGPT  
#                               low_freq_cutoff=50,   
#                               sampling_rate=1000)
# speechffr_envelope = extractor.extract_envelope()

stimulus_features = ["mel"]
stimulus_dimension = 10
features = ["eeg"] + stimulus_features
dataset_train = create_pytorch_dataset(train_generator, window_length, None,frame_tensor,process_eeg,process_stimuli,
                                  hop_length, batch_size,
                                  number_mismatch=None,
                                  data_types=(torch.float32, torch.float32),
                                  feature_dims=(64, stimulus_dimension))
dataset_train_ffr=[]
for batch in dataset_train:
    batch=list(batch)
    eeg_input=batch[0]
    mel_input=batch[1]
    extractor = EnvelopeExtractor(speech_envelope=mel_input,
                           high_freq_cutoff=4000,     
                           low_freq_cutoff=50,   
                           sampling_rate=1000)
    mel_input = extractor.extract_envelope()
    dataset_train_ffr.append((eeg_input,mel_input))
    #Shivang Add here
    #CREATE NEW DATASET WITH THIS MEL_INPUT AND THE EEG_INPUT let's call it dataset_train_ffr


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm  

class CustomResNet(nn.Module):
    def __init__(self, input_channels):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

resnet_eeg_soft = CustomResNet(64)
resnet_mel_soft = CustomResNet(10)

resnet_eeg_soft = resnet_eeg_soft.to(device)
resnet_mel_soft = resnet_mel_soft.to(device)




optimizer = optim.Adam(list(resnet_eeg_soft.parameters()) + list(resnet_mel_soft.parameters()), lr=0.001)

batch_size = 8
num_epochs = 100

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    total_loss = 0.0
    
    
    #DATASET_TRAIN_FFR IS THE NEW DATASET WITH THE FFR ENVELOPE AND THE EEG INPUT
    
    for eeg_input, mel_input in tqdm(dataset_train_ffr, desc=f'Epoch {epoch+1}/{num_epochs}'):                
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        optimizer.zero_grad()

        output_eeg = resnet_eeg_soft(eeg_input)
        output_mel = resnet_mel_soft(mel_input)

        # loss = torch.abs(output_eeg - output_mel).sum()
        loss = criterion(output_eeg,output_mel)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

torch.save(resnet_eeg.state_dict(), 'resnet_eeg_soft.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel_soft.pth')

### Resnet + Attention
Rachit please add on to the corresponding resnet layers

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
from tqdm import tqdm  

class CustomResNet(nn.Module):
    def __init__(self, input_channels):
        super(CustomResNet, self).__init__()
        self.resnet = models.resnet34(pretrained=False)
        self.resnet.conv1 = nn.Conv2d(input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.resnet.fc = nn.Linear(512, 10)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.resnet(x)
        x = self.softmax(x)
        return x

resnet_eeg_soft = CustomResNet(64)
resnet_mel_soft = CustomResNet(10)

resnet_eeg_soft = resnet_eeg_soft.to(device)
resnet_mel_soft = resnet_mel_soft.to(device)



optimizer = optim.Adam(list(resnet_eeg_soft.parameters()) + list(resnet_mel_soft.parameters()), lr=0.001)

batch_size = 8
num_epochs = 100

criterion = nn.MSELoss()

for epoch in range(num_epochs):
    total_loss = 0.0
    for eeg_input, mel_input in tqdm(dataset_train, desc=f'Epoch {epoch+1}/{num_epochs}'):
        eeg_input, mel_input = eeg_input.view(64,64,320,1).to(device) , mel_input.view(64,10,320,1).to(device)
        optimizer.zero_grad()

        output_eeg = resnet_eeg_soft(eeg_input)
        output_mel = resnet_mel_soft(mel_input)
        
        #RACHIT UPDATE THE VALUES ACCORDINGLY AND ADD THE NEXT LAYERS
        #UPDATED ATTENTION PART
        
        # input_size_eeg = 64             #change on basis of ResNet output
        # input_size_audio = 128          #change on basis of ResNet output
        # hidden_size = 128               #change on basis of ResNet input as this will be for the next ResNet layer

        # # Repeat audio data to match EEG sequence length
        # audio_data = audio_data.expand(10, -1, -1)  # Expand audio to (10, 32, 128)

        # ae = AttentionEncoder(input_size_eeg, input_size_audio, hidden_size)

        # audio_layer2 = ae(eeg_data, audio_data)
        
        #UPDATED ATTENTION PART
        
        
        #NOW FEED THE OUTPUT OF THE ATTENTION LAYER TO THE NEXT RESNET LAYER( KEEP EEG SAME, JUST CHANGE THE AUDIO INPUT)
        #APPLY ATTENTION LAYER SIMILAR TO BEFORE
        #FEED THE OUTPUT TO THE FINAL RESNET LAYER
        
       
        loss = criterion(output_eeg,output_mel)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataset_train)
    print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {average_loss}')

torch.save(resnet_eeg.state_dict(), 'resnet_eeg_soft.pth')
torch.save(resnet_mel.state_dict(), 'resnet_mel_soft.pth')